In [ ]:
#...This is the script for econometrics techniques......
# ============================================================
# Econometrics: Baseline gravity + Full specification (comparable to ML)
# - OLS on LN_EXPORT (HC1 and HAC)
# - PPML on EXPORT (HC1)
# - Train/Test evaluation metrics aligned with ML (R2, MAE, SMAPE)
# - Also contains actual vs predicted cocoa exports on the logged and raw scales for the first 20 rows
# ============================================================
import pandas as pd

data_path = "cocoa_cleaned_dataset.csv"
df = pd.read_csv(data_path)

import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.metrics import mean_absolute_error, r2_score
from IPython.display import display

# -----------------------------
# Config / path (use your path)
# -----------------------------
data_path = "cocoa_cleaned_dataset.csv"

# -----------------------------
# Load data
# -----------------------------
df = pd.read_csv(data_path)
df = df.sort_values(['PARTNER', 'YEAR']).reset_index(drop=True)

# -----------------------------
# Safe Feature Engineering
# -----------------------------
if 'LN_EXPORT' not in df.columns:
    raise ValueError("Required column 'LN_EXPORT' missing from dataset.")

df['PARTNER_ENCODED'] = df.groupby('PARTNER')['LN_EXPORT'].transform('mean')

if 'EXPORT' in df.columns:
    df['LN_EXPORT_PCT_CHANGE'] = df.groupby('PARTNER')['EXPORT'].pct_change()
    df['LN_EXPORT_PCT_CHANGE'] = np.log1p(df['LN_EXPORT_PCT_CHANGE'].fillna(0))

df['LN_EXPORT_ROLL_STD3'] = df.groupby('PARTNER')['LN_EXPORT'].rolling(3, min_periods=1).std().reset_index(0, drop=True)

lag_base = ['LN_EXPORT', 'LN_EXPORT_PCT_CHANGE', 'LN_EXPORT_ROLL_STD3',
            'LN_GDP_PARTNER', 'COCOA_PRICE', 'COCOA_PRICE_SQ', 'TEMP_C', 'SPEI', 'NDVI']

for col in lag_base:
    if col in df.columns:
        df[f'{col}_LAG1'] = df.groupby('PARTNER')[col].shift(1)
        df[f'{col}_LAG2'] = df.groupby('PARTNER')[col].shift(2)

lag_cols_created = [c for c in df.columns if c.endswith('_LAG1') or c.endswith('_LAG2')]
for col in lag_cols_created:
    df[col] = df[col].fillna(df.groupby('PARTNER')[col].transform('median'))
    if df[col].isna().any():
        df[col] = df[col].fillna(df[col].median())

if 'LN_EXPORT_LAG1' in df.columns and 'NDVI_LAG1' in df.columns:
    df['EXPORTxNDVI'] = df['LN_EXPORT_LAG1'] * df['NDVI_LAG1']
if 'LN_GDP_NIGERIA' in df.columns:
    df['GDPxPARTNER'] = df['LN_GDP_NIGERIA'] * df['PARTNER_ENCODED']

# -----------------------------
# Train/Test split
# -----------------------------
train = df[df['YEAR'] <= 2021].copy().reset_index(drop=True)
test  = df[df['YEAR'] >= 2022].copy().reset_index(drop=True)

# -----------------------------
# Feature sets
# -----------------------------
baseline_features = ['LN_GDP_NIGERIA', 'LOG_DISTANCE', 'PARTNER_ENCODED', 'ELECTION_YEARS']
baseline_features = [f for f in baseline_features if f in df.columns]

candidate_full = (
    lag_cols_created +
    ['LN_GDP_NIGERIA', 'LN_GDP_PARTNER', 'LOG_DISTANCE', 'PARTNER_ENCODED',
     'ELECTION_YEARS', 'EXPORTxNDVI', 'GDPxPARTNER',
     'PARTNER_GOV_MEAN', 'NIGERIA_GOV_MEAN',
     'COCOA_PRICE', 'COCOA_PRICE_SQ', 'TEMP_C', 'SPEI', 'NDVI']
)
full_features = [f for f in candidate_full if f in df.columns]

def add_const(dfX):
    return sm.add_constant(dfX, has_constant='add')

# -----------------------------
# Metrics helpers
# -----------------------------
def smape_raw(y_true_raw, y_pred_raw):
    a = np.abs(y_pred_raw - y_true_raw)
    denom = (np.abs(y_true_raw) + np.abs(y_pred_raw) + 1e-10)
    return 100.0 / len(y_true_raw) * np.sum(2.0 * a / denom)

def eval_metrics_logspace(name, y_tr_log, y_tr_pred_log, y_te_log, y_te_pred_log):
    return {
        "Model": name,
        "Train R2": r2_score(y_tr_log, y_tr_pred_log),
        "Test R2": r2_score(y_te_log, y_te_pred_log),
        "Train MAE": mean_absolute_error(y_tr_log, y_tr_pred_log),
        "Test MAE": mean_absolute_error(y_te_log, y_te_pred_log),
        "Train SMAPE (EXPORT%)": smape_raw(np.exp(y_tr_log), np.exp(y_tr_pred_log)),
        "Test SMAPE (EXPORT%)": smape_raw(np.exp(y_te_log), np.exp(y_te_pred_log))
    }

def eval_metrics_levelspace(name, y_tr_lvl, y_tr_pred_lvl, y_te_lvl, y_te_pred_lvl):
    return {
        "Model": name,
        "Train R2": np.nan,
        "Test R2": np.nan,
        "Train MAE (level)": mean_absolute_error(y_tr_lvl, y_tr_pred_lvl),
        "Test MAE (level)": mean_absolute_error(y_te_lvl, y_te_pred_lvl),
        "Train SMAPE (EXPORT%)": smape_raw(y_tr_lvl, y_tr_pred_lvl),
        "Test SMAPE (EXPORT%)": smape_raw(y_te_lvl, y_te_pred_lvl)
    }

# -----------------------------
# 1) Baseline OLS
# -----------------------------
metrics_list = []
coef_tables = {}

if len(baseline_features) > 0:
    X_tr_base = add_const(train[baseline_features])
    X_te_base = add_const(test[baseline_features])

    y_tr_ln = train['LN_EXPORT']
    y_te_ln = test['LN_EXPORT']

    ols_base = sm.OLS(y_tr_ln, X_tr_base).fit()
    ols_base_hc1 = ols_base.get_robustcov_results(cov_type="HC1")

    y_tr_pred_ln = ols_base.predict(X_tr_base)
    y_te_pred_ln = ols_base.predict(X_te_base)

    metrics_list.append(eval_metrics_logspace("OLS_baseline", y_tr_ln, y_tr_pred_ln, y_te_ln, y_te_pred_ln))

# -----------------------------
# 2) Baseline PPML
# -----------------------------
if len(baseline_features) > 0 and 'EXPORT' in train.columns:
    X_tr_base_lvl = add_const(train[baseline_features])
    X_te_base_lvl = add_const(test[baseline_features])

    y_tr_lvl = train['EXPORT'].clip(lower=1e-6)
    y_te_lvl = test['EXPORT'].clip(lower=1e-6)

    ppml_base = sm.GLM(y_tr_lvl, X_tr_base_lvl, family=sm.families.Poisson()).fit(cov_type="HC1")

    y_tr_ppml_pred = ppml_base.predict(X_tr_base_lvl)
    y_te_ppml_pred = ppml_base.predict(X_te_base_lvl)

    metrics_list.append(eval_metrics_levelspace("PPML_baseline", y_tr_lvl, y_tr_ppml_pred, y_te_lvl, y_te_ppml_pred))

# -----------------------------
# 3) Full specification (OLS + PPML)
# -----------------------------
if len(full_features) > 0:
    X_tr_full = add_const(train[full_features])
    X_te_full = add_const(test[full_features])

    ols_full = sm.OLS(train['LN_EXPORT'], X_tr_full).fit()
    y_tr_pred_full = ols_full.predict(X_tr_full)
    y_te_pred_full = ols_full.predict(X_te_full)
    metrics_list.append(eval_metrics_logspace("OLS_full", train['LN_EXPORT'], y_tr_pred_full, test['LN_EXPORT'], y_te_pred_full))

    if 'EXPORT' in train.columns:
        ppml_full = sm.GLM(train['EXPORT'].clip(lower=1e-6), X_tr_full, family=sm.families.Poisson()).fit(cov_type="HC1")
        y_tr_ppml_full = ppml_full.predict(X_tr_full)
        y_te_ppml_full = ppml_full.predict(X_te_full)
        metrics_list.append(eval_metrics_levelspace("PPML_full", train['EXPORT'].clip(lower=1e-6), y_tr_ppml_full,
                                                   test['EXPORT'].clip(lower=1e-6), y_te_ppml_full))

# -----------------------------
# 4) Display results
# -----------------------------
metrics_df = pd.DataFrame(metrics_list)
cols_order = [c for c in ["Model", "Train R2", "Test R2", "Train MAE", "Test MAE",
                          "Train MAE (level)", "Test MAE (level)",
                          "Train SMAPE (EXPORT%)", "Test SMAPE (EXPORT%)"] if c in metrics_df.columns]
metrics_df = metrics_df[cols_order]

print("\n📊 Econometrics Metrics (comparable with ML):")
display(metrics_df)

# -----------------------------
# 5) Actual vs Predicted (first 20 rows)
# -----------------------------
test_display = test[['YEAR', 'PARTNER', 'EXPORT', 'LN_EXPORT']].copy()

if 'y_te_pred_ln' in locals():
    test_display['PRED_OLS_baseline_LOG'] = y_te_pred_ln.values
if 'y_te_ppml_pred' in locals():
    test_display['PRED_PPML_baseline_LEVEL'] = y_te_ppml_pred.values
if 'y_te_pred_full' in locals():
    test_display['PRED_OLS_full_LOG'] = y_te_pred_full.values
if 'y_te_ppml_full' in locals():
    test_display['PRED_PPML_full_LEVEL'] = y_te_ppml_full.values

if 'PRED_OLS_baseline_LOG' in test_display.columns:
    test_display['PRED_OLS_baseline_EXPORT'] = np.exp(test_display['PRED_OLS_baseline_LOG'])
if 'PRED_OLS_full_LOG' in test_display.columns:
    test_display['PRED_OLS_full_EXPORT'] = np.exp(test_display['PRED_OLS_full_LOG'])

print("\n📈 Actual vs Predicted (test) — first 20 rows:")
display(test_display.head(20))

# -----------------------------
# Done
# -----------------------------
print("\nDone. Baseline and full-spec econometrics results successfully generated.")


In [ ]:
# ============================
# ML + SHAP + Robustness Pipeline (SMAPE on raw EXPORT scale)
# End-to-end (run in Jupyter)
# ============================
import os
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from IPython.display import display

# ML libs
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import RidgeCV, LassoCV, LinearRegression
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.inspection import permutation_importance

# Diagnostics
from statsmodels.stats.stattools import durbin_watson

# Plotting
import matplotlib.pyplot as plt

# SHAP
try:
    import shap
    HAS_SHAP = True
except Exception:
    HAS_SHAP = False
    print("⚠️ shap not installed. Install with `pip install shap` to enable SHAP analysis.")

# -----------------------------
# Paths / outputs
# -----------------------------
data_path = "cocoa_cleaned_dataset.csv"
out_dir = r".\model_outputs"
os.makedirs(out_dir, exist_ok=True)

RANDOM_STATE = 42

# -----------------------------
# 1) Load dataset
# -----------------------------
df = pd.read_csv(data_path)
df = df.sort_values(['PARTNER', 'YEAR']).reset_index(drop=True)

# -----------------------------
# 2) Feature engineering
# -----------------------------
df['LN_EXPORT_PCT_CHANGE'] = df.groupby('PARTNER')['EXPORT'].pct_change()
df['LN_EXPORT_PCT_CHANGE'] = np.log1p(df['LN_EXPORT_PCT_CHANGE'].fillna(0))

df['LN_EXPORT_ROLL_STD3'] = (
    df.groupby('PARTNER')['LN_EXPORT'].rolling(3, min_periods=1).std().reset_index(0, drop=True)
)

lag_cols = [
    'LN_EXPORT', 'LN_EXPORT_PCT_CHANGE', 'LN_EXPORT_ROLL_STD3',
    'LN_GDP_PARTNER', 'COCOA_PRICE', 'COCOA_PRICE_SQ',
    'TEMP_C', 'SPEI', 'NDVI'
]

for col in lag_cols:
    if col in df.columns:
        df[f'{col}_LAG1'] = df.groupby('PARTNER')[col].shift(1)
        df[f'{col}_LAG2'] = df.groupby('PARTNER')[col].shift(2)

# Fill NaNs in lagged features with median
for col in df.columns:
    if '_LAG' in col:
        df[col].fillna(df[col].median(), inplace=True)

# Partner encoding
partner_means = df.groupby('PARTNER')['LN_EXPORT'].mean()
df['PARTNER_ENCODED'] = df['PARTNER'].map(partner_means)

# Interaction features
if 'LN_EXPORT_LAG1' in df.columns and 'NDVI_LAG1' in df.columns:
    df['EXPORTxNDVI'] = df['LN_EXPORT_LAG1'] * df['NDVI_LAG1']
else:
    if 'EXPORTxNDVI' not in df.columns:
        df['EXPORTxNDVI'] = np.nan

if 'LN_GDP_NIGERIA' in df.columns:
    df['GDPxPARTNER'] = df['LN_GDP_NIGERIA'] * df['PARTNER_ENCODED']
else:
    if 'GDPxPARTNER' not in df.columns:
        df['GDPxPARTNER'] = np.nan

# -----------------------------
# 3) Train/Test Split
# -----------------------------
train = df[df['YEAR'] <= 2021].copy().reset_index(drop=True)
test  = df[df['YEAR'] >= 2022].copy().reset_index(drop=True)

features = [
    c for c in df.columns if '_LAG' in c or c in [
        'LN_GDP_NIGERIA', 'GDP_INTERACTION',
        'PARTNER_GOV_MEAN', 'NIGERIA_GOV_MEAN',
        'LOG_DISTANCE', 'PARTNER_ENCODED',
        'ELECTION_YEARS', 'EXPORTxNDVI', 'GDPxPARTNER'
    ]
]
features = [f for f in features if f in df.columns]
target = 'LN_EXPORT'
X_train, y_train = train[features], train[target]
X_test,  y_test  = test[features],  test[target]

# -----------------------------
# 4) SMAPE on raw EXPORT scale
# -----------------------------
def smape_raw(y_true_log, y_pred_log):
    y_true = np.expm1(y_true_log)
    y_pred = np.expm1(y_pred_log)
    return 100 / len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

# -----------------------------
# 5) ML training
# -----------------------------
tscv = TimeSeriesSplit(n_splits=5)

# Random Forest
rf_param_grid = {
    "n_estimators": [200, 400, 600],
    "max_depth": [4, 6, 8],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2", 0.8]
}
rf = RandomForestRegressor(random_state=RANDOM_STATE)
rf_grid = GridSearchCV(rf, rf_param_grid, cv=tscv, scoring="r2", n_jobs=-1, verbose=1)
rf_grid.fit(X_train, y_train)
best_rf = rf_grid.best_estimator_

train['PRED_RF'] = best_rf.predict(X_train)
test['PRED_RF']  = best_rf.predict(X_test)

# XGBoost
xgb = XGBRegressor(objective="reg:squarederror", random_state=RANDOM_STATE)
xgb_param_grid = {
    "n_estimators": [200, 400, 600],
    "max_depth": [3, 5, 6],
    "learning_rate": [0.01, 0.05, 0.1],
    "subsample": [0.7, 0.8, 1.0],
    "colsample_bytree": [0.7, 0.8, 1.0],
    "reg_alpha": [0, 0.01, 0.1],
    "reg_lambda": [1, 1.5, 2]
}
xgb_grid = GridSearchCV(xgb, xgb_param_grid, cv=tscv, scoring="r2", n_jobs=-1, verbose=1)
xgb_grid.fit(X_train, y_train)
best_xgb = xgb_grid.best_estimator_

train['PRED_XGB'] = best_xgb.predict(X_train)
test['PRED_XGB']  = best_xgb.predict(X_test)

# RidgeCV and LassoCV
ridge = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=tscv)
ridge.fit(X_train, y_train)
train['PRED_Ridge'] = ridge.predict(X_train)
test['PRED_Ridge']  = ridge.predict(X_test)

lasso = LassoCV(alphas=[0.01, 0.1, 1.0], cv=tscv, max_iter=5000)
lasso.fit(X_train, y_train)
train['PRED_Lasso'] = lasso.predict(X_train)
test['PRED_Lasso']  = lasso.predict(X_test)

# Ensemble
weights = {"RF": 0.4, "XGB": 0.4, "Ridge": 0.2}
train['PRED_ENS'] = train['PRED_RF']*weights["RF"] + train['PRED_XGB']*weights["XGB"] + train['PRED_Ridge']*weights["Ridge"]
test['PRED_ENS']  = test['PRED_RF']*weights["RF"] + test['PRED_XGB']*weights["XGB"] + test['PRED_Ridge']*weights["Ridge"]

# -----------------------------
# 6) Evaluation
# -----------------------------
def evaluate(model_name, y_train, y_train_pred, y_test, y_test_pred):
    return {
        "Model": model_name,
        "Train R²": r2_score(y_train, y_train_pred),
        "Test R²": r2_score(y_test, y_test_pred),
        "Train MAE": mean_absolute_error(y_train, y_train_pred),
        "Test MAE": mean_absolute_error(y_test, y_test_pred),
        "Train SMAPE": smape_raw(y_train, y_train_pred),
        "Test SMAPE": smape_raw(y_test, y_test_pred)
    }

results = []
results.append(evaluate("RF",    y_train, train['PRED_RF'],    y_test, test['PRED_RF']))
results.append(evaluate("XGB",   y_train, train['PRED_XGB'],   y_test, test['PRED_XGB']))
results.append(evaluate("Ridge", y_train, train['PRED_Ridge'], y_test, test['PRED_Ridge']))
results.append(evaluate("Lasso", y_train, train['PRED_Lasso'], y_test, test['PRED_Lasso']))
results.append(evaluate("ENS",   y_train, train['PRED_ENS'],   y_test, test['PRED_ENS']))

results_df = pd.DataFrame(results)
results_df["R2_Gap"] = results_df["Train R²"] - results_df["Test R²"]

print("\n📊 Model Comparison:")
display(results_df)
results_df.to_csv(os.path.join(out_dir, "ml_model_comparison.csv"), index=False)

# -----------------------------
# 7) Rolling-window CV (SMAPE raw)
# -----------------------------
years = sorted(df['YEAR'].unique())
cv_results = []
for year in years[5:]:
    train_cv = df[df['YEAR'] < year]
    test_cv  = df[df['YEAR'] == year]
    if train_cv.empty or test_cv.empty:
        continue
    X_tr, y_tr = train_cv[features], train_cv[target]
    X_te, y_te = test_cv[features], test_cv[target]
    try:
        best_rf.fit(X_tr, y_tr)
        y_pred_rf = best_rf.predict(X_te)
        cv_results.append({
            "YEAR": year,
            "R2_RF": r2_score(y_te, y_pred_rf),
            "SMAPE_RF": smape_raw(y_te, y_pred_rf)
        })
    except Exception:
        continue

cv_df = pd.DataFrame(cv_results)
print("\n🔄 Rolling-window CV results:")
display(cv_df)
cv_df.to_csv(os.path.join(out_dir, "rolling_cv_summary.csv"), index=False)

# -----------------------------
# 8) Feature importances
# -----------------------------
print("\n🔥 Full RF Feature Importances:")
rf_importances = pd.Series(best_rf.feature_importances_, index=features).sort_values(ascending=False)
display(rf_importances)
rf_importances.to_csv(os.path.join(out_dir, "rf_feature_importances.csv"))

print("\n🔥 Full XGB Feature Importances:")
xgb_importances = pd.Series(best_xgb.feature_importances_, index=features).sort_values(ascending=False)
display(xgb_importances)
xgb_importances.to_csv(os.path.join(out_dir, "xgb_feature_importances.csv"))

# -----------------------------
# 9) Residuals & Bias
# -----------------------------
test['RESID_RF']    = y_test - test['PRED_RF']
test['RESID_XGB']   = y_test - test['PRED_XGB']
test['RESID_Ridge'] = y_test - test['PRED_Ridge']
test['RESID_Lasso'] = y_test - test['PRED_Lasso']
test['RESID_ENS']   = y_test - test['PRED_ENS']

print("\n📉 Residual Mean (Bias Check):")
display(test[['RESID_RF', 'RESID_XGB', 'RESID_Ridge', 'RESID_Lasso', 'RESID_ENS']].mean())

# -----------------------------
# 10) SHAP Analysis
# -----------------------------
if HAS_SHAP:
    explainer_rf = shap.TreeExplainer(best_rf)
    explainer_xgb = shap.TreeExplainer(best_xgb)

    shap_values_rf = explainer_rf.shap_values(X_test)
    shap_values_xgb = explainer_xgb.shap_values(X_test)

    shap_imp_rf = pd.DataFrame({
        "feature": X_test.columns,
        "mean_abs_shap": np.abs(shap_values_rf).mean(axis=0)
    }).sort_values("mean_abs_shap", ascending=False).reset_index(drop=True)

    shap_imp_xgb = pd.DataFrame({
        "feature": X_test.columns,
        "mean_abs_shap": np.abs(shap_values_xgb).mean(axis=0)
    }).sort_values("mean_abs_shap", ascending=False).reset_index(drop=True)

    print("\n🔥 SHAP (RF) mean |SHAP| table (full ranking):")
    display(shap_imp_rf)
    shap_imp_rf.to_csv(os.path.join(out_dir, "shap_rf_meanabs.csv"), index=False)

    print("\n🔥 SHAP (XGB) mean |SHAP| table (full ranking):")
    display(shap_imp_xgb)
    shap_imp_xgb.to_csv(os.path.join(out_dir, "shap_xgb_meanabs.csv"), index=False)

    # Visualizations
    plt.figure(figsize=(10,6))
    shap.summary_plot(shap_values_rf, X_test, plot_type="bar", show=False)
    plt.title("RF – Mean SHAP Importance")
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, "shap_rf_bar.png"))
    plt.show()

    plt.figure(figsize=(10,6))
    shap.summary_plot(shap_values_xgb, X_test, plot_type="bar", show=False)
    plt.title("XGB – Mean SHAP Importance")
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, "shap_xgb_bar.png"))
    plt.show()

else:
    print("⚠️ SHAP not available — skip SHAP steps.")

# -----------------------------
# 11) Robustness checks
# -----------------------------
# 11.1 Naive Lag-only baseline
naive_results = None
if 'LN_EXPORT_LAG1' in train.columns:
    lr_naive = LinearRegression()
    Xn_train = train[['LN_EXPORT_LAG1']].fillna(0)
    Xn_test  = test[['LN_EXPORT_LAG1']].fillna(0)
    lr_naive.fit(Xn_train, train[target])
    y_tr_naive = lr_naive.predict(Xn_train)
    y_te_naive = lr_naive.predict(Xn_test)
    naive_results = {
        "Model": "Naive (Lag1)",
        "Train R²": r2_score(train[target], y_tr_naive),
        "Test R²": r2_score(test[target], y_te_naive),
        "Train MAE": mean_absolute_error(train[target], y_tr_naive),
        "Test MAE": mean_absolute_error(test[target], y_te_naive),
        "Train SMAPE": smape_raw(train[target], y_tr_naive),
        "Test SMAPE": smape_raw(test[target], y_te_naive)
    }
    print("\n🔰 Naïve benchmark (Lag1):")
    display(pd.DataFrame([naive_results]))

# 11.2 Permutation importance
perm_importance_tables = {}
for name, model in [("RF", best_rf), ("XGB", best_xgb)]:
    try:
        perm = permutation_importance(model, X_train, y_train, n_repeats=20, random_state=RANDOM_STATE, n_jobs=-1)
        perm_df = pd.DataFrame({
            "feature": X_train.columns,
            "importance_mean": perm.importances_mean,
            "importance_std": perm.importances_std
        }).sort_values("importance_mean", ascending=False).reset_index(drop=True)
        perm_importance_tables[name] = perm_df
        print(f"\n🔁 Permutation importance ({name}):")
        display(perm_df)
        perm_df.to_csv(os.path.join(out_dir, f"perm_importance_{name}.csv"), index=False)
    except Exception as e:
        print(f"Permutation importance failed for {name}: {e}")

# 11.3 Durbin-Watson on residuals
dw = {}
try:
    dw['RF'] = durbin_watson(test['RESID_RF'].dropna())
    dw['XGB'] = durbin_watson(test['RESID_XGB'].dropna())
    dw['Ridge'] = durbin_watson(test['RESID_Ridge'].dropna())
    print("\n📈 Durbin-Watson statistics (test residuals):")
    display(pd.Series(dw).to_frame("DurbinWatson"))
except Exception as e:
    print("Durbin-Watson failed:", e)

# 11.4 Bootstrapped metrics
def bootstrap_metrics_on_test(model, X_test_df, y_test_ser, n_boot=200, random_state=RANDOM_STATE):
    rng = np.random.RandomState(random_state)
    rows = X_test_df.index.values
    out = []
    for i in range(n_boot):
        idx = rng.choice(rows, size=len(rows), replace=True)
        Xi = X_test_df.loc[idx]
        yi = y_test_ser.loc[idx]
        pred = model.predict(Xi)
        out.append({
            "R2": r2_score(yi, pred),
            "MAE": mean_absolute_error(yi, pred),
            "SMAPE": smape_raw(yi, pred)
        })
    return pd.DataFrame(out)

bootstrap_summary = {}
for name, model in [("RF", best_rf), ("XGB", best_xgb), ("Ridge", ridge)]:
    try:
        bdf = bootstrap_metrics_on_test(model, X_test, y_test, n_boot=200)
        bootstrap_summary[name] = bdf.describe().loc[['mean','std']]
        print(f"\n🎲 Bootstrap summary ({name}) — mean & std of metrics on test resamples:")
        display(bdf.describe().loc[['mean','std']])
        bdf.to_csv(os.path.join(out_dir, f"bootstrap_metrics_{name}.csv"), index=False)
    except Exception as e:
        print(f"Bootstrap failed for {name}: {e}")

# Save main ML results
results_df.to_csv(os.path.join(out_dir, "ml_model_comparison.csv"), index=False)
print("\nDone. All ML + SHAP (if installed) + robustness outputs saved to", out_dir)
